In [1]:

import dotenv

# https://python.langchain.com/docs/use_cases/question_answering/quickstart
# Get the API key from the user 'input' 
# import getpass
# import os
# os.environ["OPENAI_API_KEY"] = getpass.getpass()
# dot env ref:
# https://stackoverflow.com/questions/40216311/reading-in-environment-variables-from-an-environment-file

dotenv.load_dotenv("./.env")


True

In [2]:
from langchain_openai import ChatOpenAI

# Models: 
# https://platform.openai.com/docs/models/gpt-3-5-turbo
llm = ChatOpenAI(model="gpt-3.5-turbo")

llm


ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001EC4A316620>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001EC4A340190>, openai_api_key=SecretStr('**********'), openai_proxy='')

In [3]:
import asyncio
import logging
from typing import Dict, List, Optional, Sequence, Callable

from langchain_core.documents import Document

from langchain_community.document_loaders.base import BaseLoader

logger = logging.getLogger(__name__)


class ExMongodbLoader(BaseLoader):
    """Load MongoDB documents."""

    def __init__(
        self,
        connection_string: str,
        db_name: str,
        collection_name: str,
        *,
        filter_criteria: Optional[Dict] = None,
        field_names: Optional[Sequence[str]] = None,
        doc_transformer: Callable = str,
    ) -> None:
        try:
            from motor.motor_asyncio import AsyncIOMotorClient
        except ImportError as e:
            raise ImportError(
                "Cannot import from motor, please install with `pip install motor`."
            ) from e
        if not connection_string:
            raise ValueError("connection_string must be provided.")

        if not db_name:
            raise ValueError("db_name must be provided.")

        if not collection_name:
            raise ValueError("collection_name must be provided.")

        self.client = AsyncIOMotorClient(connection_string)
        self.db_name = db_name
        self.collection_name = collection_name
        self.filter_criteria = filter_criteria or {}
        self.field_names = field_names or []
        self.doc_transformer = doc_transformer

        self.db = self.client.get_database(db_name)
        self.collection = self.db.get_collection(collection_name)

    def load(self) -> List[Document]:
        """Load data into Document objects.

        Attention:

        This implementation starts an asyncio event loop which
        will only work if running in a sync env. In an async env, it should
        fail since there is already an event loop running.

        This code should be updated to kick off the event loop from a separate
        thread if running within an async context.
        """
        return asyncio.run(self.aload())

    async def aload(self) -> List[Document]:
        """Load data into Document objects."""
        result = []
        total_docs = await self.collection.count_documents(self.filter_criteria)

        # Construct the projection dictionary if field_names are specified
        projection = (
            {field: 1 for field in self.field_names} if self.field_names else None
        )

        async for doc in self.collection.find(self.filter_criteria, projection):
            metadata = {
                "database": self.db_name,
                "collection": self.collection_name,
            }

            # Extract text content from filtered fields or use the entire document
            text = self.doc_transformer(doc)

            result.append(Document(page_content=text, metadata=metadata))

        if len(result) != total_docs:
            logger.warning(
                f"Only partial collection of documents returned. "
                f"Loaded {len(result)} docs, expected {total_docs}."
            )

        return result


In [4]:

# document loaders
# https://python.langchain.com/docs/modules/data_connection/document_loaders/
# https://python.langchain.com/docs/integrations/document_loaders/mongodb

# add this import for running in jupyter notebook
import nest_asyncio

nest_asyncio.apply()

# IT IS TOO WEAK:
# from langchain_community.document_loaders.mongodb import MongodbLoader

def transform_detail_doc(doc: dict) -> str:
    return f'名称：{doc.get("项目名称", "")}; 简介：{doc.get("项目信息", {}).get("项目简介", "")}'


loader = ExMongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="final",
    collection_name="itemDetail",
    doc_transformer=transform_detail_doc,
    field_names="项目信息 项目名称".split(),
)
# KeyError: '项目信息.项目简介'

In [5]:
title_and_abstracts = loader.load()

len(title_and_abstracts)


247965


- 250000 docs
- 20 char/doc
- 5 token/char
- 0.002 rmb/k*token



In [14]:
print(250000*20*5*0.002/1000, "rmb")

50.0 rmb


In [6]:

# 一个文档似乎有点太长了
# 加入简介是否必要？
# 直接切分也不太合适，会出现只有简介的部分，对大模型选题造成干扰

title_and_abstracts[0]


Document(page_content='名称：高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例; 简介：随着我国高等教育事业的发展，高等院校越来越多，学校招生人数以及老师人数越来越多，从相关资料了解到高校师生人数在成急速增长趋势，可见，我国高校数量增多，校园人员增长迅速，校园人口密度越来越大。高密度的人口必然带来高的能源消耗，在对所高等学校进行能源统计及结果测算时发现全国高等院校总能耗约占全国生活能耗的7%，是全国人均生活用能4倍；的高校总用水量占全国总用水量的8.14%，是全国人均用水量的1.9倍。从而可见高校能源消耗的特点为能耗总数量大，人均能耗大。根据相关学者在2008年对上海高校能耗调查发现高校的建筑能耗在高校总能耗中占83.2%的比例。  综合高校建筑能耗高的问题，本项目的主要研究目的是对广州地区建筑进行调查研究，结合建筑规划设计，构造技术和建筑材料使用状况的调研以及通过生态大师软件的模拟就广东白云学院校园既有建筑隔热性能进行实测分析,并对常用建筑节能措施的综合效益进行理论探讨。同时综合考虑和设计出合理的改造方案，探讨出适宜的节能改造的措施。期望通过本项目的广东白云学院的案例研究结论能为对湿热地区高校校园建筑的设计以及节能改造项目起到某种参考作用。 并且通过研究希望寻找出采用低成本高效益的适宜的建筑节能技术措施，创造舒适而健康的室内热建筑是本次项目的主要目的。', metadata={'database': 'final', 'collection': 'itemDetail'})

In [7]:


loader = ExMongodbLoader(
    connection_string="mongodb://localhost:27017/",
    db_name="final",
    collection_name="itemDetail",
    field_names= ["项目名称"],
    doc_transformer=lambda x:str(x.get("项目名称", ""))
)
titles = loader.load()
titles = titles[:32]



In [11]:

chars_cnt = 0

for doc in titles:
    text = doc.page_content
    chars_cnt += len(text)

print(chars_cnt/len(titles))


17.96875


In [8]:
[print(x.page_content) for x in titles[:8]]

高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例
大学生对中西方传统节日价值取向的研究
“语伞”雨伞共享平台设计
互联网+大学城闲散物流资源跨界整合创新实践
太阳跟随系统
移动快充跨境电商创业
静态平衡仪
疏勒河流域水-能源-粮食纽带关系研究


[None, None, None, None, None, None, None, None]

In [15]:
from langchain_community.embeddings import QianfanEmbeddingsEndpoint

from langchain_elasticsearch import ElasticsearchStore

# https://python.langchain.com/docs/modules/data_connection/text_embedding/
# https://python.langchain.com/docs/integrations/text_embedding/baidu_qianfan_endpoint/

# https://python.langchain.com/docs/modules/data_connection/vectorstores/
# https://python.langchain.com/docs/integrations/vectorstores/elasticsearch/



embedding = QianfanEmbeddingsEndpoint(model="Embedding-V1")

res = embedding.embed_documents([
    "你好，世界！", 
    "Hello, world!",
    "国内链接OpenAI不太方便，于是我使用百度公司的model。",
])


[INFO] [04-05 15:19:22] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


In [26]:
[print(f"{num:.2f}", end=", ") for num in res[0]]
print()
[print(f"{num:.2f}", end=", ") for num in res[1]]
print()
[print(f"{num:.2f}", end=", ") for num in res[-1]]
type(res[0])

import numpy as np

arrays = [np.array(ls) for ls in res]

print()
print(np.dot(arrays[0], arrays[1]))
print(np.dot(arrays[0], arrays[2]))
print(np.dot(arrays[1], arrays[2]))


-0.02, -0.04, -0.03, 0.10, -0.03, -0.05, 0.01, -0.05, 0.03, 0.01, 0.04, -0.13, 0.08, -0.02, -0.07, -0.02, 0.00, -0.00, 0.03, 0.02, -0.01, -0.04, -0.05, -0.02, 0.03, 0.03, 0.08, 0.05, 0.02, -0.01, -0.00, 0.02, 0.03, -0.06, -0.02, 0.05, 0.04, 0.02, -0.08, 0.04, -0.01, 0.05, -0.00, 0.09, 0.03, -0.06, -0.04, -0.08, -0.08, -0.05, 0.00, 0.01, -0.02, 0.08, -0.01, -0.04, -0.11, -0.02, 0.07, -0.06, 0.01, 0.06, 0.08, -0.06, 0.06, 0.01, -0.00, -0.00, 0.07, 0.01, -0.02, -0.03, 0.06, 0.08, -0.06, 0.04, -0.03, 0.00, -0.00, -0.02, 0.06, -0.08, 0.09, -0.07, -0.08, -0.01, 0.02, 0.04, -0.02, -0.01, -0.01, -0.08, 0.02, -0.08, -0.08, -0.04, 0.02, -0.01, -0.01, -0.07, 0.07, 0.06, 0.07, 0.00, 0.08, -0.05, -0.07, -0.11, 0.01, 0.04, -0.02, 0.05, 0.02, -0.01, -0.05, 0.08, 0.07, -0.02, 0.02, -0.05, -0.07, 0.08, -0.02, 0.11, -0.05, 0.06, -0.12, -0.04, -0.05, 0.06, 0.01, 0.01, 0.07, -0.01, -0.03, 0.00, 0.03, -0.02, -0.04, 0.09, -0.10, -0.02, -0.02, 0.07, 0.05, 0.09, 0.05, -0.01, -0.03, 0.03, -0.03, 0.01, -0.02, 0

In [31]:
elastic_vector_store = ElasticsearchStore(
    es_url="http://localhost:9200",
    index_name="rag_test",
    embedding=embedding,
)

elastic_vector_store.client.delete_by_query(query={
    "match_all": {}
}, index="rag_test", ignore=[400, 404],)

elastic_vector_store = ElasticsearchStore.from_documents(
    documents=titles,
    embedding=embedding,
    index_name="rag_test",
    es_url="http://localhost:9200",
)


C:\Users\realj\AppData\Local\Temp\ipykernel_32572\187219858.py:7: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  elastic_vector_store.client.delete_by_query(query={
[INFO] [04-05 15:45:58] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


[INFO] [04-05 15:45:59] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


In [36]:
retriever = elastic_vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 5})

retrieved_docs = retriever.invoke("大学生平台")

print(len(retrieved_docs))
[print(doc.page_content) for doc in retrieved_docs]


[INFO] [04-05 15:47:05] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


5
互联网+大学城闲散物流资源跨界整合创新实践
高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例
“海上丝绸之路﹒中国史迹”申报世界文化遗产背景下台山山咀码头、三洲港及沿港临街环境整治设计
大学毕业生面试服装设计与服饰搭配咨询服务工作室
大学生对中西方传统节日价值取向的研究


[None, None, None, None, None]

In [37]:
retrieved_docs = retriever.invoke("创新研究啊")

print(len(retrieved_docs))
[print(doc.page_content) for doc in retrieved_docs]

[INFO] [04-05 15:47:29] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


5
刻纸艺术在室内陈设中的传承与创新
高热导事故容错核燃料芯块的高温高压制备与研究
多旋翼飞行器电网巡检相关问题的研究
盆栽行业新型树屋的造景研发
智能除臭马桶优化设计


[None, None, None, None, None]

In [39]:
from langchain_community.chat_models import QianfanChatEndpoint
from langchain_core.language_models.chat_models import HumanMessage

# https://python.langchain.com/docs/integrations/chat/
# https://python.langchain.com/docs/integrations/chat/baidu_qianfan_endpoint/

llm = QianfanChatEndpoint(model="ERNIE-3.5-8K", streaming=True)
messages = [HumanMessage(content="你好，我是Jayden")]
llm.invoke(messages)


[WARNING] [04-05 16:16:23] base.py:406 [t:29384]: retry is not available when stream is enabled
[INFO] [04-05 16:16:23] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /chat/completions


AIMessage(content='你好，Jayden！有什么我可以帮助你的吗？', response_metadata={'token_usage': {}, 'model_name': 'ERNIE-3.5-8K', 'finish_reason': 'stop'}, id='run-855959a1-b19a-4460-84de-9603ce48fa94-0')

In [41]:
from langchain_core.prompts import PromptTemplate

template = """你是大学生创新创业指导中心的专业教授，请你根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：{student_info}
往年选题：{history_titles}
拟定新的选题："""

rag_prompt = PromptTemplate.from_template(template)


In [46]:
ex_stu_info = "信息管理与信息系统专业，自然语言处理方向"

ex_msg = rag_prompt.invoke(
    {
        "student_info": ex_stu_info,
        "history_titles": ";".join([
            doc.page_content for doc in
            retriever.invoke(ex_stu_info, {"k": 5})
        ])
    }
).to_messages()

print(ex_msg[0].content)

[INFO] [04-05 16:28:07] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /embeddings/embedding-v1


你是大学生创新创业指导中心的专业教授，请你根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：信息管理与信息系统专业，自然语言处理方向
往年选题：基于深度学习理论的城市轨道交通智能视频监控与报警系统;基于机器学习的可拼接智能挂钟系统研发;太阳跟随系统;大学毕业生面试服装设计与服饰搭配咨询服务工作室;高校校园建筑能耗模拟及节能改造设计---以广东白云学院为例
拟定新的选题：


In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return ";".join(doc.page_content for doc in docs)


rag_chain = (
    {
        "history_titles": retriever | format_docs, 
        "student_info": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [50]:
for chunk in rag_chain.stream("信息管理与信息系统专业，自然语言处理方向，数学建模比赛经历"):
    print(chunk, end="", flush=True)

[INFO] [04-05 16:36:09] openapi_requestor.py:336 [t:23356]: requesting llm api endpoint: /embeddings/embedding-v1


[WARNING] [04-05 16:36:10] base.py:406 [t:29384]: retry is not available when stream is enabled
[INFO] [04-05 16:36:10] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /chat/completions


考虑到学生的专业背景、自然语言处理方向的兴趣以及数学建模比赛的经历，我为学生拟定了以下几个合适的选题：

1. **基于自然语言处理的智能客服系统研发**


	* **背景**：随着人工智能技术的发展，智能客服系统在企业中的应用越来越广泛。结合自然语言处理技术，可以实现更加智能、高效的客户服务。
	* **研究内容**：研究并应用自然语言处理技术，如文本分类、情感分析、实体识别等，构建一个能够准确理解并回复客户咨询的智能客服系统。通过数学建模优化系统的性能，提高客户满意度。
2. **基于深度学习的社交媒体情感分析**


	* **背景**：社交媒体上产生了大量的用户评论和反馈，通过情感分析可以了解用户的真实想法和意见。
	* **研究内容**：利用深度学习技术，如循环神经网络（RNN）或长短期记忆网络（LSTM），对社交媒体上的文本数据进行情感分析。通过数学建模方法，提高情感分析的准确性和效率。
3. **基于自然语言处理的智能文本摘要生成系统**


	* **背景**：在信息爆炸的时代，如何快速获取关键信息成为了一个重要的问题。智能文本摘要生成系统可以帮助用户快速浏览和理解文本内容。
	* **研究内容**：结合自然语言处理技术，研究并开发一个能够自动生成文本摘要的系统。利用数学建模方法，优化摘要生成的质量和效率。
4. **基于机器学习的在线教育平台个性化推荐系统**


	* **背景**：在线教育平台的发展迅速，如何为用户提供个性化的学习资源推荐是一个关键问题。
	* **研究内容**：利用机器学习技术，如协同过滤、内容推荐等，构建一个能够为用户推荐合适学习资源的个性化推荐系统。通过数学建模方法，提高推荐的准确性和用户满意度。

这些选题结合了学生的专业背景、兴趣方向和数学建模经验，旨在通过实践和研究，提高学生的创新能力和解决问题的能力。

In [60]:
new_template = """你是大学生创新创业指导中心的专业教授，请你根据学生情况和往年选题，为学生拟定几个合适的选题。
学生情况：{student_info}
往年选题：{history_titles}
注意：严禁重复往年选题内容，**无需解释拟定选题的理由**，一行一个直接列出选题。
拟定新的选题："""

new_rag_prompt = PromptTemplate.from_template(new_template)


In [61]:
new_rag_chain = (
    {
        "history_titles": retriever | format_docs, 
        "student_info": RunnablePassthrough(),
    }
    | new_rag_prompt
    | llm
    | StrOutputParser()
)
for chunk in new_rag_chain.stream("信息管理与信息系统专业，自然语言处理方向，数学建模比赛经历"):
    print(chunk, end="", flush=True)

[INFO] [04-05 16:39:25] openapi_requestor.py:336 [t:15308]: requesting llm api endpoint: /embeddings/embedding-v1


[WARNING] [04-05 16:39:25] base.py:406 [t:29384]: retry is not available when stream is enabled
[INFO] [04-05 16:39:25] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /chat/completions


1. 基于自然语言处理的在线评论情感分析系统
2. 智能问答系统设计与实现
3. 社交媒体用户行为分析与预测
4. 基于知识图谱的信息推荐系统
5. 多模态情感识别技术在人机交互中的应用

In [62]:
for chunk in new_rag_chain.stream("会计学专业，企业税务与决策项目经历，资本结构优化方向"):
    print(chunk, end="", flush=True)

[INFO] [04-05 16:42:42] openapi_requestor.py:336 [t:32104]: requesting llm api endpoint: /embeddings/embedding-v1
[WARNING] [04-05 16:42:43] base.py:406 [t:29384]: retry is not available when stream is enabled
[INFO] [04-05 16:42:43] openapi_requestor.py:336 [t:29384]: requesting llm api endpoint: /chat/completions


1. 基于大数据的企业税务风险预测与防控策略
2. 资本结构优化下的企业财务决策支持系统设计与实现
3. 跨国企业税务筹划与合规性管理研究
4. 基于财务视角的中小企业成长策略研究
5. 会计准则变化对企业财务报表的影响分析